In [1]:
import pandas as pd
import trino
import json

# Cấu hình để Pandas hiển thị đầy đủ nội dung
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

print("Các thư viện đã sẵn sàng.")

# Kết nối tới Trino service
conn = trino.dbapi.connect(
    host='localhost',
    port=8080,
    user='python-client', 
    catalog='lakehouse', # Catalog Iceberg
    schema='rva'         # Database
)

cursor = conn.cursor()
print(f"Kết nối Trino (Catalog: {conn.catalog}, Schema: {conn.schema}) thành công!")

Các thư viện đã sẵn sàng.
Kết nối Trino (Catalog: lakehouse, Schema: rva) thành công!


In [2]:
cursor.execute("SHOW TABLES FROM lakehouse.rva")
tables = cursor.fetchall()

print("Các bảng có trong schema 'rva':")
print(tables)

# Kết quả mong đợi: [['bronze_raw'], ['silver_detections']]

Các bảng có trong schema 'rva':
[['bronze_raw'], ['silver_detections']]


In [3]:
# Xem schema của bảng silver_detections
df_schema_silver = pd.read_sql("DESCRIBE lakehouse.rva.silver_detections", conn)
df_schema_silver

/tmp/ipykernel_70235/2116236874.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_schema_silver = pd.read_sql("DESCRIBE lakehouse.rva.silver_detections", conn)


,Column,Type,Extra,Comment
0,schema_version,varchar,,
1,pipeline_run_id,varchar,,
2,store_id,varchar,,
3,camera_id,varchar,,
4,frame_index,bigint,,
5,capture_ts,timestamp(6),,
6,img_w,integer,,
7,img_h,integer,,
8,det_id,varchar,,
9,class_name,varchar,,


In [4]:
# Đếm tổng số dòng (records) trong bảng silver
cursor.execute("SELECT COUNT(*) FROM lakehouse.rva.silver_detections")
total_silver_rows = cursor.fetchone()[0]

print(f"Tổng số dòng (frames) trong 'bronze_raw': 410") # Con số từ notebook trước
print(f"Tổng số dòng (detections) trong 'silver_detections': {total_silver_rows}")

Tổng số dòng (frames) trong 'bronze_raw': 410
Tổng số dòng (detections) trong 'silver_detections': 586


In [5]:
# Đọc 10 dòng đầu tiên của bảng silver
df_sample_silver = pd.read_sql("SELECT * FROM lakehouse.rva.silver_detections LIMIT 10", conn)
df_sample_silver

/tmp/ipykernel_70235/2767393899.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sample_silver = pd.read_sql("SELECT * FROM lakehouse.rva.silver_detections LIMIT 10", conn)


,schema_version,pipeline_run_id,store_id,camera_id,frame_index,capture_ts,img_w,img_h,det_id,class_name,class_id,conf,bbox_x1,bbox_y1,bbox_x2,bbox_y2,track_id,processing_ts
0,1.0,04e852b3b933499ea9523264aee7a8f8,store_01,cam_01,1,2025-11-13 06:25:47.484,1280,720,1-0,person,0,0.8199,345,58,438,430,-1,2025-11-14 03:29:19.514000+00:00
1,1.0,04e852b3b933499ea9523264aee7a8f8,store_01,cam_01,2,2025-11-13 06:25:47.614,1280,720,2-0,person,0,0.8254,344,58,438,430,-1,2025-11-14 03:29:19.518000+00:00
2,1.0,04e852b3b933499ea9523264aee7a8f8,store_01,cam_01,3,2025-11-13 06:25:47.734,1280,720,3-0,person,0,0.8267,344,58,438,429,1,2025-11-14 03:29:19.519000+00:00
3,1.0,04e852b3b933499ea9523264aee7a8f8,store_01,cam_01,8,2025-11-13 06:25:48.390,1280,720,8-0,person,0,0.8213,343,57,438,431,1,2025-11-14 03:29:19.522000+00:00
4,1.0,04e852b3b933499ea9523264aee7a8f8,store_01,cam_01,9,2025-11-13 06:25:48.521,1280,720,9-0,person,0,0.8178,343,57,438,431,1,2025-11-14 03:29:19.523000+00:00
5,1.0,04e852b3b933499ea9523264aee7a8f8,store_01,cam_01,10,2025-11-13 06:25:48.650,1280,720,10-0,person,0,0.8246,343,57,436,431,1,2025-11-14 03:29:19.523000+00:00
6,1.0,04e852b3b933499ea9523264aee7a8f8,store_01,cam_01,4,2025-11-13 06:25:47.866,1280,720,4-0,person,0,0.8237,344,57,437,430,1,2025-11-14 03:29:19.519000+00:00
7,1.0,04e852b3b933499ea9523264aee7a8f8,store_01,cam_01,5,2025-11-13 06:25:47.989,1280,720,5-0,person,0,0.8251,344,58,437,431,1,2025-11-14 03:29:19.520000+00:00
8,1.0,04e852b3b933499ea9523264aee7a8f8,store_01,cam_01,6,2025-11-13 06:25:48.117,1280,720,6-0,person,0,0.8244,343,58,437,431,1,2025-11-14 03:29:19.520000+00:00
9,1.0,04e852b3b933499ea9523264aee7a8f8,store_01,cam_01,7,2025-11-13 06:25:48.261,1280,720,7-0,person,0,0.8212,344,57,437,431,1,2025-11-14 03:29:19.521000+00:00


In [6]:
# Kiểm tra xem các giá trị 'null' đã được thay thế chưa
df_clean_check = pd.read_sql("""
    SELECT 
        class_name,
        track_id,
        COUNT(*) as total
    FROM lakehouse.rva.silver_detections
    GROUP BY 1, 2
    ORDER BY 3 DESC
""", conn)

print("Kiểm tra các giá trị đã được làm sạch (class_name và track_id):")
df_clean_check

/tmp/ipykernel_70235/2526840184.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_clean_check = pd.read_sql("""


Kiểm tra các giá trị đã được làm sạch (class_name và track_id):


,class_name,track_id,total
0,person,1,409
1,person,5,145
2,person,7,15
3,person,-1,10
4,person,4,7


In [7]:
# Kiểm tra giá trị NULL và giá trị rỗng ('')
df_clean_check = pd.read_sql("""
    SELECT 
        CASE 
            WHEN class_name IS NULL THEN 'NULL_VALUE'
            WHEN class_name = '' THEN 'EMPTY_STRING'
            ELSE class_name 
        END AS class_name_status,
        
        CASE
            WHEN track_id = -1 THEN 'CLEANED_TRACK_ID'
            WHEN track_id IS NULL THEN 'NULL_TRACK_ID'
            ELSE 'VALID_TRACK_ID'
        END AS track_id_status,
        
        COUNT(*) as total
    FROM lakehouse.rva.silver_detections
    GROUP BY 1, 2
    ORDER BY 3 DESC
""", conn)

print("Kiểm tra chi tiết giá trị 'class_name' và 'track_id':")
df_clean_check

/tmp/ipykernel_70235/944284329.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_clean_check = pd.read_sql("""


Kiểm tra chi tiết giá trị 'class_name' và 'track_id':


,class_name_status,track_id_status,total
0,person,VALID_TRACK_ID,576
1,person,CLEANED_TRACK_ID,10
